This notebook is used to populate the DB with stock data.

In [1]:
""" Import all the necessary packages and smf modules and check if database connection works.
"""

import time
import pandas as pd
from smf.db_engine import DbEngine
import eikon as ek
import numpy as np
import datetime as dt

# try:
#     with open(f"smf\eikon_app_key.json") as ek_app_key:
#             ek_app_key = json.load(ek_app_key)
#     ek.set_app_key(ek_app_key["key"])
# except:
#     ek.set_app_key("d14a6a14480842328de35bc001771c605538616a")    
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')
db = DbEngine()

-I- Successful database connection


In [2]:
""" Create table "tickers_and_classifications" which contains details on the economic sector, business sector
    and industry group for each given RIC.
"""
sectors_table_name = "tickers_and_classifications"
db_data = db.fetch_table("ticker_symbols")
tickers_list = db_data["ric"].values.tolist()
eikon_data = ek.get_data(tickers_list, ["TR.TRBCEconomicSector","TR.TRBCBusinessSector", "TR.TRBCIndustryGroup"])

sectors_df = pd.DataFrame(data=eikon_data[0])
sectors_df.columns = ["ric", "economic_sector", "business_sector", "industry_group"]
sectors_df = sectors_df.set_index("ric")

db.append_df_as_row(sectors_df, sectors_table_name, if_exists="replace", index=True)
db.set_primary_key(sectors_table_name, "ric")

-W- Table ticker_symbols not found


TypeError: 'NoneType' object is not subscriptable

In [3]:
""" Create table "volatilies_as_at_{date}" which contains the annualized standard deviation
    of the relative price change for the n most recent trading days, expressed as a percentage.
"""
tickers_list = db.fetch_table("ticker_symbols")["ric"].values.tolist()
volatilities_days = [2, 5, 10, 20, 25, 30, 40, 50, 60, 80, 90, 100, 120, 150, 160, 180, 200, 240, 250, 260]
volatilities_fields = list(map(lambda num:f"TR.Volatility{str(num)}D", volatilities_days))
eikon_data, err = ek.get_data(tickers_list, volatilities_fields)
volatilities_df = pd.DataFrame(eikon_data)
volatilities_df.columns = ["ric"] + list(map(lambda num:f"{str(num)}_day_volatility", volatilities_days))
volatilities_df = volatilities_df.set_index("ric")

today = dt.datetime.now().strftime("%Y_%m_%d")
volatilities_table_name = f"volatilities_as_at_{today}"
db.append_df_as_row(volatilities_df, volatilities_table_name, if_exists="replace", index=True)
db.set_primary_key(volatilities_table_name, "ric")

-W- Table ticker_symbols not found


TypeError: 'NoneType' object is not subscriptable

In [ ]:
""" This is the most problematic part...
    For each stock/ticker/RIC, create a table containing the following information for that stock:
        - Closing prices
        - PE ratio
        - PB ratio
        - PFCF ratio
        - EV-EBITDA ratio
""" 

time_ = time.time()
tickers_list= db.fetch_table("ticker_symbols")["ric"].values.tolist()
start = dt.date(2016, 6, 30).strftime("%Y-%m-%d")
end = dt.datetime.today().strftime("%Y-%m-%d")


problem_stocks = []

for (count, ticker) in reversed(list(enumerate(tickers_list))):
    try:
        # Closing prices
        ek_closing_prices = ek.get_timeseries(ticker, ["Close"], start_date=start, end_date=end)
        ticker_info_df = pd.DataFrame(ek_closing_prices)
        ticker_info_df.columns = ["close_price"]

        # Price-earnings
        date_field = f"TR.HistPE(SDate={start},EDate={end},Period=FI0,Frq=FI).date"
        field = f"TR.HistPE(SDate={start},EDate={end},Period=FI0,Frq=FI)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Historic P/E"]).set_index("Date")
        if data_df["Historic P/E"].isna().all():
            ticker_info_df["price_earnings"] = np.nan
        else:
            data_df.columns = ["price_earnings"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df = ticker_info_df.fillna(method="ffill")

        # Price-to-book
        date_field = f"TR.PriceToBVPerShare(SDate={start},EDate={end},Frq=D).date"
        field = f"TR.PriceToBVPerShare(SDate={start},EDate={end},Frq=D)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Price To Book Value Per Share (Daily Time Series Ratio)"]).set_index("Date")
        if data_df["Price To Book Value Per Share (Daily Time Series Ratio)"].isna().all():
            ticker_info_df["price_to_book"] = np.nan
        else:
            data_df.columns = ["price_to_book"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df["price_to_book"] = ticker_info_df["price_to_book"].fillna(method="ffill")

        # Price-to-free-cashflow
        date_field = f"TR.HistPricetoFreeOperatingCashFlowperShareAvgDilutedSharesOut(SDate={start},EDate={end},Period=FI0,Frq=FI).date"
        field = f"TR.HistPricetoFreeOperatingCashFlowperShareAvgDilutedSharesOut(SDate={start},EDate={end},Period=FI0,Frq=FI)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Historic Price/FOCF/Shr (dil.)"]).set_index("Date")
        if data_df["Historic Price/FOCF/Shr (dil.)"].isna().all():
            ticker_info_df["price_to_fcf"] = np.nan
        else:
            data_df.columns = ["price_to_fcf"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df["price_to_fcf"] = ticker_info_df["price_to_fcf"].fillna(method="ffill")

        # EV-EBITDA
        date_field = f"TR.HistEnterpriseValueEBITDA(SDate={start},EDate={end},Period=FY0,Frq=FY).date"
        field = f"TR.HistEnterpriseValueEBITDA(SDate={start},EDate={end},Period=FY0,Frq=FY)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Historic EV/EBITDA, FY"]).set_index("Date")
        if data_df["Historic EV/EBITDA, FY"].isna().all():
            ticker_info_df["ev_ebitda"] = np.nan
        else:
            data_df.columns = ["ev_ebitda"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df["ev_ebitda"] = ticker_info_df["ev_ebitda"].fillna(method="ffill")

#         # Current ratio
#         date_field = f"TR.CurrentRatio(SDate={start},EDate={end},Frq=Y).date"
#         field = f"TR.CurrentRatio(SDate={start},EDate={end},Frq=Y)"
#         eik_data, err = ek.get_data(ticker, [date_field, field])
#         data_df = pd.DataFrame(eik_data, columns=["Date", "Current Ratio"]).set_index("Date")
#         data_df.columns = ["current_ratio"]
#         data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
#         ticker_info_df = ticker_info_df.join(data_df)
#         ticker_info_df["current_ratio"] = ticker_info_df["current_ratio"].fillna(method="ffill")

#         # Quick Ratio
#         date_field = f"TR.QuickRatio(SDate={start},EDate={end},Frq=Y).date"
#         field = f"TR.QuickRatio(SDate={start},EDate={end},Frq=Y)"
#         eik_data, err = ek.get_data(ticker, [date_field, field])
#         data_df = pd.DataFrame(eik_data, columns=["Date", "Quick Ratio"]).set_index("Date")
#         data_df.columns = ["quick_ratio"]
#         data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
#         ticker_info_df = ticker_info_df.join(data_df)
#         ticker_info_df["quick_ratio"] = ticker_info_df["quick_ratio"].fillna(method="ffill")

#         # Dollar Turnover
#         date_field = f"TR.TURNOVER(SDate={start},EDate={end},Frq=D).date"
#         field = f"TR.TURNOVER(SDate={start},EDate={end},Frq=D)"
#         eik_data, err = ek.get_data(ticker, [date_field, field])
#         data_df = pd.DataFrame(eik_data, columns=["Date", "Turnover"]).set_index("Date")
#         data_df.columns = ["dollar_turnover"]
#         data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
#         ticker_info_df = ticker_info_df.join(data_df)
        
        # Because duplicate rows arise, we can't create SQL primary keys until we get rid of these duplicates...
        ticker_info_df = ticker_info_df.loc[~ticker_info_df.index.duplicated(keep='first')]
        
        # Since '.' has a specific function in SQL, we use '_' instead.
        db.append_df_as_row(ticker_info_df, ticker.replace(".","_"), if_exists="replace", index=True) 
        db.set_primary_key(ticker.replace(".","_"), "Date")
        
        print(f"-I- Successfully stored table for ({count}) {ticker}. {count-1} more to go.")

    except Exception as e:
        problem_stocks.append(ticker)
        print(f"-W- Could not store table for ({count}) {ticker}. Error message: {str(e)}")


print("\n",f"Whole thing took {time.time() - time_} seconds")

In [ ]:
print(problem_stocks)

I used the cell below to remove stocks that could not be updated. I used the year as the criteria: if the latest row is from year 2019, then the table was not updated. Otherwise, it would be 2020.

In [ ]:
string = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = \"SMFQuantDB\""
stocks = pd.read_sql(string, db.engine).iloc[0:2466]
response = []

for count, stock in enumerate(stocks.values.flatten()):
    string = "SELECT MAX(DATE) FROM " + stock
    date = pd.to_datetime(pd.read_sql(string, db.engine).values[0][0])
    if date.date().year == 2019:
        db.delete_table(stock)

In [ ]:
test_date = np.datetime64('2019-11-15')
pd.to_datetime(test_date).date().year == 2019

# Testing out volatility calculator:

In [ ]:
price = db.fetch_data_series("close_price")

In [ ]:
volatilities = db.fetch_table("volatilities_as_at_2020_01_24").set_index("ric")
volatilities.index = [ric.replace(".", "_") for ric in volatilities.index]

"... from the standard deviation of day to day logarithmic historical price changes... the annualized standard deviation of the relative price change for the 60 most recent trading days closing price, expressed as a percentage."

In [ ]:
time_ = time.time()
tickers_list= ["PTLA.OQ"]
start = dt.date(2016, 6, 30).strftime("%Y-%m-%d")
end = dt.datetime.today().strftime("%Y-%m-%d")

problem_stocks = []

for (count, ticker) in reversed(list(enumerate(tickers_list))):
    try:
        # Closing prices
        ek_closing_prices = ek.get_timeseries(ticker, ["Close"], start_date=start, end_date=end)
        ticker_info_df = pd.DataFrame(ek_closing_prices)
        ticker_info_df.columns = ["close_price"]
        ticker_info_df = ticker_info_df.loc[~ticker_info_df.index.duplicated(keep='first')]
        print(f"-I- Successfully stored table for ({count}) {ticker}. {count-1} more to go.")

    except Exception as e:
        problem_stocks.append(ticker)
        print(f"-W- Could not store table for ({count}) {ticker}. Error message: {str(e)}")


print("\n",f"Whole thing took {time.time() - time_} seconds")

In [ ]:
def volatility(period, as_at, price_table):
    business_days_per_year = 224.5819722273368 # This number was obtained by comparing with the annualized historical
                                               # volatility given by Eikon on the DIB for TR.Volatility60D, etc.
    as_at = pd.Timestamp(as_at)
    # Calculate log-returns
    returns = price_table.dropna().pct_change() + 1
    log_returns = returns.transform(np.log)
    
    # Calculate standard deviation of log-returns (as a percentage)
    log_returns = pd.DataFrame(log_returns.loc[:as_at,:].iloc[-60:])#* np.sqrt(business_days_per_year))
    volatilities = log_returns.std() * 100
    
    # Rename headings/column name
    volatilities.index.name = "ric"
    volatilities.columns = [f"{period}_day_volatility"]
    
    return(log_returns)

In [ ]:
period = 60
as_at = "2020-01-24"
new = volatility(period, as_at, price)

In [ ]:
test = pd.DataFrame(index=new.index)
test["new"] = new
test["old"] = volatilities.loc[:,"60_day_volatility"]
test["ratio"] = (test["old"] / test["new"])**2
test["ratio"].describe()

In [ ]:
new

In [ ]:
test.loc["PTLA_OQ",:]

In [ ]:
ptla_price = db.fetch_data_series("close_price")

In [ ]:
ptla_price = ptla_price.loc[:,"PTLA_OQ"]

In [ ]:
ptla_price.tail()

In [ ]:
ptla_price.loc[:,ptla_price.isna().all()]